# Linear Regression

## 1- Creating Linear Regression Class Structure

In [302]:
import numpy as np

class LinearRegression():
    def __init__(self, fit_method='ols', loss_function="rmse", l1=0, l2=0, learning_rate=0.01, epochs=1000, min_step_size=0.001, gradient_descent='batch', batch_size=32):
        """
        Initialize the LinearRegression model with a specified fitting method.

        Parameters:
        - fit_method: The fitting method to use: "ols" for Ordinary Least Squares, "gd" for Gradient Descent.
        - learning_rate: Learning rate for Gradient Descent.
        - loss_function: Loss function to use. rmse for Root Mean Squared Error. Only Root Mean Squared Error is supported for now.
        - l1: L1 regularization parameter.
        - l2: L2 regularization parameter.
        - epochs: Number of epochs for Gradient Descent.
        - min_step_size: Minimum step size for Gradient Descent.
        - gradient_descent: Type of gradient_descent. Possible values: "batch", "stochastic", "mini-batch". 
        - batch_size: Size of batch for mini-bactch gradient descent.

        Notes: 
        - You cant use l1 regularization with ols because there is no closed form solution.
        """

        # general parameters
        self.fit_method = fit_method
        self.loss_function = loss_function

        # regularization parameters
        self.l1 = l1
        self.l2 = l2

        # gradient descent parameters
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.min_step_size = min_step_size
        self.gradient_descent = gradient_descent
        self.batch_size = batch_size

        # initialize weights to none
        self.weights = None # W0 will be bias.
    
    def calculate_loss(self, y_true, y_pred):
        """
        Calculate the loss function value for the given true and predicted values with respect to loss function type and regularization parameters.

        Parameters:
        - y_true: True target values.
        - y_pred: Predicted target values.
        """

        if self.loss_function == 'rmse':
            loss = np.sqrt(np.mean((y_true - y_pred) ** 2)) + self.l1 * np.sum(np.abs(self.weights)) + self.l2 * np.sum(self.weights ** 2)
        else:
            raise ValueError("loss_function should be either 'mse' or 'mae'")

        return loss
    
    def calculate_gradient(self, X, y):
        """
        Calculate the gradient for the loss function for given X, y_true and y_pred values.

        Parameters:
        - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
        - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
        """

        y_pred = self._predict(X)

        if self.loss_function == 'rmse':
            loss_gradient = - X.T @ (y - y_pred) / (X.shape[0] * np.sqrt(np.mean((y - y_pred) ** 2))) + self.l1 * np.sign(self.weights) + 2 * self.l2 * self.weights
        else:
            raise ValueError("loss_function should be rmse.")

        return loss_gradient

    def fit_ols(self, X, y):
        pass

    def fit_gd(self, X, y):
        pass

    def fit_gd_batch(self, X, y):
        pass

    def fit_gd_stochastic(self, X, y):
        pass

    def fit_gd_mini_batch(self, X, y):
        pass

    def fit(self, X, y):
        """
        Fit the model to the data based on selected fit method.

        Parameters:
        - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
        - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
        """

        # Add bias terms coefficent to the X for easier bias term handling.
        X = np.c_[np.ones((X.shape[0], 1)), X]

        if self.fit_method == 'ols':
            self.fit_ols(X, y)
        elif self.fit_method == 'gd':
            self.fit_gd(X, y)
        else:
            raise ValueError("fit_method should be either 'ols' or 'gd'")


    def predict(self, X):
        """
        Predict the target values for given inputs.

        Parameters:
        - X: Input value array for prediction. Should be numpy array with shape (n_samples, n_features).

        Returns:
        - y: Predictions values for input array X. numpy array with shape (n_samples, )
        """

        if self.weights is None:
            raise ValueError("Model has not been fitted yet.")
        
        # Add bias terms coefficent to the X for prediction.
        X = np.c_[np.ones((X.shape[0], 1)), X]

        y = self._predict(X)
        return y
    
    def _predict(self, X):
        """
        Helper method for gradient descent. Using self.predict add 1s for the biases.
        """
        return X @ self.weights

## 2- Fit Method Implementations

### A- Ordinary Least Squares

This part is taken from Ian Goodfellow, Yoshua Bengio, Aaron Courville - Deep Learning-The MIT Press (2016).\
Given the gradient of the training Mean Squared Error (MSE):

$$
\nabla_w \text{MSE}_{\text{train}} = 0 \tag{5.6}
$$

This implies:

$$
\nabla_w \left( \frac{1}{m} \| \hat{y}^{(\text{train})} - y^{(\text{train})} \|^2_2 \right) = 0 \tag{5.7}
$$

Expanding it:

$$
\frac{1}{m} \nabla_w \| X^{(\text{train})} w - y^{(\text{train})} \|^2_2 = 0 \tag{5.8}
$$

Taking the gradient with respect to \( w \):

$$
\nabla_w \left( X^{(\text{train})} w - y^{(\text{train})} \right)^{\top} \left( X^{(\text{train})} w - y^{(\text{train})} \right) = 0 \tag{5.9}
$$

This simplifies to:

$$
\nabla_w \left( w^{\top} X^{(\text{train})^{\top}} X^{(\text{train})} w - 2 w^{\top} X^{(\text{train})^{\top}} y^{(\text{train})} + y^{(\text{train})^{\top}} y^{(\text{train})} \right) = 0 \tag{5.10}
$$

Setting the gradient to zero:

$$
2 X^{(\text{train})^{\top}} X^{(\text{train})} w - 2 X^{(\text{train})^{\top}} y^{(\text{train})} = 0 \tag{5.11}
$$

Solving for \( w \):

$$
w = \left( X^{(\text{train})^{\top}} X^{(\text{train})} \right)^{-1} X^{(\text{train})^{\top}} y^{(\text{train})} \tag{5.12}
$$


In [303]:
def fit_ols(self, X, y):
    """
    Fit the model to the data using ordinary least squares fit method by calculating weights by given formula.

    Parameters:
    - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
    - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
    """

    self.weights = np.linalg.inv(X.T @ X + self.l2 * np.identity(X.shape[1])) @ X.T @ y

# Assign it to the class method
LinearRegression.fit_ols = fit_ols

### B- Gradient Descent

#### a- Batch gradient descent

In [304]:
def fit_gd_batch(self, X, y):
    """
    Fit the model to the data using batch gradient descent method by updating weights untill convergence.
    Batch gradients use all the training data for updating weights at each step.

    Parameters:
    - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
    - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
    """

    # Initialize weights
    self.weights = np.random.randn(X.shape[1], ) * 0.01
    self.weights[0] = 0 # Thats what they do in NN
    
    # Gradient Descent Loop
    for _ in range(self.epochs):
        gradient = self.calculate_gradient(X, y)
        self.weights = self.weights - self.learning_rate * gradient

# Assign it to the class method
LinearRegression.fit_gd_batch = fit_gd_batch

#### b- Stochastic gradient descent

In [305]:
def fit_gd_stochastic(self, X, y):
    """
    Fit the model to the data using batch gradient descent method by updating weights untill convergence.
    Batch gradients use all the training data for updating weights at each step.

    Parameters:
    - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
    - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
    """

    # Initialize weights
    self.weights = np.random.randn(X.shape[1], ) * 0.01
    self.weights[0] = 0 # Thats what they do in NN
    
    n = X.shape[0]
    current_index = 0
    for epoch in range(self.epochs):
        if epoch % n == 0:
            indices = np.arange(n)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]
        
        current_X, current_y = X[current_index : current_index + 1], y[current_index]
        current_index = (current_index + 1) % n
        gradient = self.calculate_gradient(current_X, current_y)
        self.weights = self.weights - self.learning_rate * gradient

# Assign it to the class method
LinearRegression.fit_gd_stochastic = fit_gd_stochastic

#### c- Mini-batch gradient descent

In [306]:
def fit_gd_mini_batch(self, X, y):
    """
    Fit the model to the data using batch gradient descent method by updating weights untill convergence.
    Batch gradients use all the training data for updating weights at each step.

    Parameters:
    - X: Input value array for training data. Should be numpy array with shape (n_samples, n_features).
    - y: Target value array for training data. Should be numpy array with shape (n_samples, ).
    """

    # Initialize weights
    self.weights = np.random.randn(X.shape[1], ) * 0.01
    self.weights[0] = 0 # Thats what they do in NN

    n = X.shape[0]
    current_index = 0
    for epoch in range(self.epochs):
        if epoch % n == 0:
            indices = np.arange(n)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]
        
        current_X, current_y = X[current_index : min(current_index + self.batch_size, n)], y[current_index : min(current_index + self.batch_size, n)]
        current_index = min(current_index + self.batch_size, n) % n
        gradient = self.calculate_gradient(current_X, current_y)
        self.weights = self.weights - self.learning_rate * gradient

# Assign it to the class method
LinearRegression.fit_gd_mini_batch = fit_gd_mini_batch

#### d- Merge all together

In [307]:
def fit_gd(self, X, y):
    if self.gradient_descent == 'batch':
        self.fit_gd_batch(X, y)
    elif self.gradient_descent == 'stochastic':
        self.fit_gd_stochastic(X, y)
    elif self.gradient_descent == 'mini-batch':
        self.fit_gd_mini_batch(X, y)
    else:
        raise ValueError("Incorrect gradient_descent value. Possible values: batch, stochastic, mini-batch.")
    
# Assign it to the class method
LinearRegression.fit_gd = fit_gd

## 3- Testing Linear Regression

### A- Import and Split Dataset

In [308]:
# Import dataset
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


boston = fetch_openml(name="boston", version=1, as_frame=True)

X = np.array(boston.data).astype(float)
y = np.array(boston.target).astype(float)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize train test
X_train_mean = X_train.mean(axis=0)
X_train_std = X_train.std(axis=0)

X_train_normalized = (X_train - X_train_mean) / X_train_std
X_test_normalized = (X_test - X_train_mean) / X_train_std

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X.shape, y.shape

((404, 13), (404,), (102, 13), (102,), (506, 13), (506,))

### B- Test OLS

#### a- Without Regularization

In [309]:
from sklearn.metrics import root_mean_squared_error

ols_model = LinearRegression(fit_method="ols", loss_function="rmse")
ols_model.fit(X_train_normalized, y_train)

ols_pred_train = ols_model.predict(X_train_normalized)
ols_rmse_train = root_mean_squared_error(y_train, ols_pred_train)

ols_pred_test = ols_model.predict(X_test_normalized)
ols_rmse_test = root_mean_squared_error(y_test, ols_pred_test)

print(ols_model.weights)
print("OLS training rmse: ", ols_rmse_train)
print("OLS test rmse: ", ols_rmse_test)

[22.79653465 -1.00213533  0.69626862  0.27806485  0.7187384  -2.0223194
  3.14523956 -0.17604788 -3.0819076   2.25140666 -1.76701378 -2.03775151
  1.12956831 -3.61165842]
OLS training rmse:  4.6520331848801675
OLS test rmse:  4.928602182665336


#### c- L2 Regularization

In [310]:
ols_l2_model = LinearRegression(fit_method="ols", loss_function="rmse", l2=1)
ols_l2_model.fit(X_train_normalized, y_train)

ols_l2_pred_train = ols_l2_model.predict(X_train_normalized)
ols_l2_rmse_train = root_mean_squared_error(y_train, ols_l2_pred_train)

ols_l2_pred_test = ols_l2_model.predict(X_test_normalized)
ols_l2_rmse_test = root_mean_squared_error(y_test, ols_l2_pred_test)

print(ols_l2_model.weights)
print("OLS with L2 regularization training rmse: ", ols_l2_rmse_train)
print("OLS with L2 regularization test rmse: ", ols_l2_rmse_test)

[22.74024691 -0.99218679  0.6777488   0.2522143   0.72248078 -1.99083465
  3.15157218 -0.17726162 -3.04502895  2.17324941 -1.69555879 -2.02783351
  1.127197   -3.59897667]
OLS with L2 regularization training rmse:  4.652517478423405
OLS with L2 regularization test rmse:  4.933847995363045


### C- Test Gradient Descent

#### a- Batch gradient descent

In [311]:
batch_gd_model = LinearRegression(fit_method="gd", loss_function="rmse", gradient_descent='batch', epochs=10000)
batch_gd_model.fit(X_train_normalized, y_train)

batch_gd_pred_train = batch_gd_model.predict(X_train_normalized)
batch_gd_rmse_train = root_mean_squared_error(y_train, batch_gd_pred_train)

batch_gd_pred_test = batch_gd_model.predict(X_test_normalized)
batch_gd_rmse_test = root_mean_squared_error(y_test, batch_gd_pred_test)

print(batch_gd_model.weights)
print("Batch gradient descent training rmse: ", batch_gd_rmse_train)
print("Batch gradient descent test rmse: ", batch_gd_rmse_test)

[22.7965344  -0.94343025  0.56883539  0.07435118  0.7470226  -1.90017958
  3.21093155 -0.19545468 -2.96708916  1.63719496 -1.11664882 -2.00144587
  1.1295799  -3.58772585]
Batch gradient descent training rmse:  4.658384167500506
Batch gradient descent test rmse:  4.975195483288953


#### b- Stochastic gradient descent

In [312]:
stoc_gd_model = LinearRegression(fit_method="gd", loss_function="rmse", gradient_descent='stochastic', epochs=10000)
stoc_gd_model.fit(X_train_normalized, y_train)

stoc_gd_pred_train = stoc_gd_model.predict(X_train_normalized)
stoc_gd_rmse_train = root_mean_squared_error(y_train, stoc_gd_pred_train)

stoc_gd_pred_test = stoc_gd_model.predict(X_test_normalized)
stoc_gd_rmse_test = root_mean_squared_error(y_test, stoc_gd_pred_test)

print(stoc_gd_model.weights)
print("Stochastic gradient descent training rmse: ", stoc_gd_rmse_train)
print("Stochastic gradient descent test rmse: ", stoc_gd_rmse_test)

[21.92       -1.20788957  0.6041245   0.26417322  0.30123354 -0.89498717
  4.2379923  -1.08926357 -2.1422088   0.69671495 -0.87866294 -1.4433898
  1.24435545 -2.11143699]
Stochastic gradient descent training rmse:  4.953760561452936
Stochastic gradient descent test rmse:  5.40076021642857


#### c- Mini-batch gradient descent

In [313]:
minibatch_gd_model = LinearRegression(fit_method="gd", loss_function="rmse", gradient_descent='mini-batch', epochs=10000, batch_size=32)
minibatch_gd_model.fit(X_train_normalized, y_train)

minibatch_gd_pred_train = minibatch_gd_model.predict(X_train_normalized)
minibatch_gd_rmse_train = root_mean_squared_error(y_train, minibatch_gd_pred_train)

minibatch_gd_pred_test = minibatch_gd_model.predict(X_test_normalized)
minibatch_gd_rmse_test = root_mean_squared_error(y_test, minibatch_gd_pred_test)

print(minibatch_gd_model.weights)
print("Stochastic gradient descent training rmse: ", minibatch_gd_rmse_train)
print("Stochastic gradient descent test rmse: ", minibatch_gd_rmse_test)

[22.70491275 -0.93568033  0.53116471  0.07266191  0.59393139 -1.73026602
  3.45927782 -0.405116   -2.85411164  1.46806366 -1.19012204 -1.93893251
  1.10722116 -3.26944264]
Stochastic gradient descent training rmse:  4.6730044174981025
Stochastic gradient descent test rmse:  5.025684219881524
